In [ ]:
import matplotlib
import sys
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import time
import scipy as sp
from scipy import ndimage

start_time=time.time()


#Number of Montecarlo sweeps for calculation
sweeps = 1000  
glidersweeps = 200
glidersweepsvel= 20

#Ask for array length
N=50

def initialstateglider(N):
    initstate=np.zeros((N,N))
    initstate[25,25]=1 
    initstate[24,25]=1 
    initstate[23,24]=1 
    initstate[25,24]=1 
    initstate[25,23]=1 
 
    return initstate
    
def initialstateoscillator(N):
    initstate=np.zeros((N,N))
    initstate[25,25]=1 
    initstate[25,26]=1
    initstate[26,25]=1
    initstate[25,24]=1
    initstate[24,25]=1
    initstate[27,25]=1
    initstate[23,25]=1
    initstate[22,25]=1
    initstate[28,25]=1
    initstate[25,27]=1
    initstate[25,28]=1
    initstate[25,23]=1
    initstate[25,22]=1
    initstate[26,23]=1
    initstate[24,23]=1
    initstate[27,24]=1
    initstate[27,26]=1
    initstate[26,27]=1
    initstate[24,27]=1
    initstate[23,26]=1
    initstate[23,24]=1
    
    return initstate

#Generate random initial state
def randominitialstate(N):   
    
    initstate = np.random.randint(2, size=(N,N))
    return initstate

def calculateCoM(cell):

    CoM =sp.ndimage.measurements.center_of_mass(cell)

    return CoM

def calculateTime(cell):
    
    activeCells= np.sum(cell)
    return activeCells

def nextState(cell):
    newcell=np.zeros((N,N))
    for a in range(N):
        for b in range(N):
            
            neighbours = cell[(a+1)%N,b] + cell[a,(b+1)%N] + cell[(a-1)%N,b] + cell[a,(b-1)%N] + cell[(a+1)%N,(b+1)%N] + cell[(a+1)%N,(b-1)%N] + cell[(a-1)%N, (b-1)%N] + cell[(a-1)%N, (b+1)%N]
            
            if cell[a,b] == 1:
                
                if neighbours < 2:
                    newcell[a,b]=0
                elif neighbours > 3:
                    newcell[a,b]=0
            
                else:
                    newcell[a,b]=1
            else:
                if neighbours == 3:
                    newcell[a,b]=1

    cell = newcell

    return cell
            
initialarray=input("Type r for random, g for glider and o for oscillator:" "")


if initialarray == "r": 
    
    cell = randominitialstate(N)
    
    Plotactivesteps=np.zeros(sweeps)
    
    im=plt.imshow(cell,animated=True)
    
    for i in range(sweeps):         
        
        cell = nextState(cell)
        Plotactivesteps[i] = calculateTime(cell)
        
        plt.cla()
        im=plt.imshow(cell,animated=True)
        plt.draw()
        plt.pause(0.0001)
    
       
    f = plt.figure(figsize=(18, 10));   
    plt.scatter(np.arange(sweeps),Plotactivesteps, s=50, marker='o', color='IndianRed')
    plt.title("Equilibration time an array of " + str(N*N) + " cells and "+ str(sweeps)+ " Montecarlo sweeps")
    plt.xlabel("Sweeps", fontsize=20);
    plt.ylabel("Number of active sites", fontsize=20);         plt.axis('tight');     

elif initialarray == "o":
    
    cell= initialstateoscillator(N)
    
    Plotactivesteps=np.zeros(sweeps)
    
    im=plt.imshow(cell,animated=True)
    
    for i in range(sweeps):         
        
        cell = nextState(cell)
        Plotactivesteps[i] = calculateTime(cell)
        
        plt.cla()
        im=plt.imshow(cell,animated=True)
        plt.draw()
        plt.pause(0.0001)
    
       
    g = plt.figure(figsize=(18, 10));   
    plt.scatter(np.arange(sweeps),Plotactivesteps, s=50, marker='o', color='IndianRed')
    plt.title("Equilibration time an array of " + str(N*N) + " cells and "+ str(sweeps)+ " Montecarlo sweeps")
    plt.xlabel("Sweeps", fontsize=20);
    plt.ylabel("Number of active sites", fontsize=20);         plt.axis('tight');   

elif initialarray == "g":
    
    cell= initialstateglider(N)
    
    Plotactivesteps=np.zeros(sweeps)
    
    CoMx=np.zeros(glidersweepsvel)
    CoMy=np.zeros(glidersweepsvel)

    
    
    im=plt.imshow(cell,animated=True)
    
    for i in range(glidersweepsvel):         
        
        cell = nextState(cell)
        Plotactivesteps[i] = calculateTime(cell)
    

        if abs(max(np.where(cell==1)[0])-min(np.where(cell==1)[0])) < (N/2) and abs(max(np.where(cell==1)[1])-min(np.where(cell==1)[1])) < (N/2): 
            
            CoMx[i]=calculateCoM(cell)[0]
            CoMy[i]=calculateCoM(cell)[1]
        
        
        plt.cla()
        im=plt.imshow(cell,animated=True)
        plt.draw()
        plt.pause(0.0001)
    
    Vx = np.polyfit(np.arange(glidersweepsvel),CoMx,1)[0]
    Vy = np.polyfit(np.arange(glidersweepsvel),CoMy,1)[0]
    
    CoMx=[]
    CoMy=[]
    sweepsPlot=[]
    glid=np.arange(glidersweeps)
    
    for i in range(glidersweeps):         
        
        cell = nextState(cell)
        Plotactivesteps[i] = calculateTime(cell)
    

        if abs(max(np.where(cell==1)[0])-min(np.where(cell==1)[0])) < (N/2) and abs(max(np.where(cell==1)[1])-min(np.where(cell==1)[1])) < (N/2): 
            
            CoMx.append(calculateCoM(cell)[0])
            CoMy.append(calculateCoM(cell)[1])
            sweepsPlot.append(glid[i])

        plt.cla()
        im=plt.imshow(cell,animated=True)
        plt.draw()
        plt.pause(0.0001)
    
    
    
    centx = plt.figure(figsize=(18, 10));   
    plt.scatter(sweepsPlot,CoMx, s=50, marker='o', color='IndianRed')
    plt.title("Position of the Center of Mass x coordinate against number of sweeps")
    plt.xlabel("Sweeps", fontsize=20);
    plt.ylabel("Center of mass x coordinate", fontsize=20);         plt.axis('tight');

    centy= plt.figure(figsize=(18, 10));
    plt.scatter(sweepsPlot,CoMy, s=50, marker='o', color='IndianRed')
    plt.title("Position of the Center of Mass x coordinate against number of sweeps")
    plt.xlabel("Sweeps", fontsize=20);
    plt.ylabel("Center of mass y coordinate", fontsize=20);         plt.axis('tight');

    print("The velocity of the glider is: " + str(np.sqrt(Vx**2 + Vy**2)))

else:
    
    print("You did not print o, g nor r, please try again")



print("It took: "+str((time.time()-start_time)/60)+" minutes to run")